# **Este *ipynb* está enmarcado dentro de un trabajo que busca predecir qué alumnos del departamento de Sistemas la UTN FRBA desertarán.**

Los datos disponibles fueron unificados en una sola tabla y se removieron los registros con valores nulos.

En este **ipynb** se utilizarán herramientas del aprendizaje estadístico para crear un modelo supervisado que prediga el outcome de la variable 'deserto'.

## **Importación de librerías**

In [ ]:
## Importamos librerías para manipulación de datos.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
## Importamos librerías de aprendizaje automático.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Importamos librerías para poder crear Pipelines.
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Importamos librerias de PCA.
from sklearn.decomposition import PCA

## **Google Colaboratory o Local**
El notebook podrá ser corrido tanto localmente como en Google Colaboratory.

El usuario deberá modificar el root path de acuerdo a su conveniencia.

In [ ]:
## Verificamos si estamos corriendo el noteboock en Google Colaboratory.
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)

## En el caso de estar en Google Colab, montamos nuestro Drive.
if var_google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  ## Direccion root donde está el notebook.
  root_path = "/content/gdrive/MyDrive/Colab Notebooks/GIAR/"

## En el caso de no estar en Google Colab, estamos corriendo localmente el notebook.
else:
  root_path = ""

## **Dataset**


In [ ]:
## Importamos el dataset.
df = pd.read_csv(root_path + 'datos/base_datos_estudiantes02_00.csv')

## **PREPOCESAMIENTO**

In [ ]:
df.columns

In [ ]:
## Imprimimos las dimensiones del dataset.
print(f'Dimensiones del dataset: {df.shape}')

In [ ]:
## Calculamos el Sample to Feature Ratio (S2FR) y lo imprimimos.
S2FR = df.shape[0] / (df.shape[1]-2)
S2FR

In [ ]:
## Dividimos el dataset en la variable dependiente "y", en este caso "deserto", y las independientes "x".
y = np.array(df[["deserto"]])
x = df.drop(['Codigo Alumno','deserto'], axis=1)

In [ ]:
x.shape

In [ ]:
## Diferenciamos las variables numéricas de las categóricas.
num_features = ['edad al ingreso',
                'Ciclo Lectivo de Cursada',
                'Cantidad de veces recursada regular',
                'Descripción de recursada regular_No Recurso', 
                'Descripción de recursada regular_Recurso 1 Vez', 
                'Descripción de recursada regular_Recurso 2 Veces', 
                'Descripción de recursada regular_Recurso 3 Veces', 
                'Descripción de recursada regular_Recurso 4 Veces', 
                'Descripción de recursada regular_Recurso 5 Veces',
                'noAprobado', 
                'Aprobado', 
                'Promociono', 
                'Nota', 
                'Nota_max_prom', 
                'Indice_aprobacion', 
                'Turno_Mañana', 
                'Turno_Noche', 
                'Turno_Tarde', 
                'Tipo de aprobación_Cambio Curso', 
                'Tipo de aprobación_Firmo', 
                'Tipo de aprobación_Libre', 
                'Tipo de aprobación_No Firmo', 
                'Tipo de aprobación_Promociono',
                'cantidad de años']

cat_features = ['EsTecnico', 'Sexo','grupo_ingreso_nivel1']

In [ ]:
## Separamos el dataset en train y test. 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=824, random_state=4)

In [ ]:
xtrain.shape

In [ ]:
xtest.shape

In [ ]:
pd.DataFrame(ytrain).value_counts(normalize = True)

In [ ]:
pd.DataFrame(ytest).value_counts(normalize = True)

In [ ]:
## Utilizamos la técnica de oversampling para balancear los datos de train.
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
xtrain_over, ytrain_over = oversample.fit_resample(xtrain, ytrain)

In [ ]:
xtrain_over.shape

In [ ]:
pd.DataFrame(ytrain_over).value_counts(normalize=True)

In [ ]:
xtrain_over.head()

In [ ]:
## Definimos las transformaciones a realizar para las variables numéricas. 
## Las estandarizaremos utilizando StandardScaler.
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

## Definimos la transformación a realizar para las variables categóricas.
do_nothing = FunctionTransformer(lambda x: x)

## Definimos el ColumnTransformer que será ejecutado al momento del fitting.
preprocesamiento = ColumnTransformer(
    transformers=[ 
        ("num", numeric_transformer, num_features),
        ("cat", do_nothing, cat_features)
    ]
)

In [ ]:
## Matriz X al aplicar el preprosesamiento definido.
normalize_xtrain_over = pd.DataFrame(preprocesamiento.fit_transform(xtrain_over))

In [ ]:
normalize_train_over = pd.concat([pd.DataFrame(ytrain_over),normalize_xtrain_over], axis=1)

In [ ]:
normalize_xtrain_over.shape

In [ ]:
normalize_train_over.head()

In [ ]:
## Calculamos la matriz de correlación lineal de Pearson.
corr = normalize_train_over.corr()
# Imprimimos el heatmap.
fig = plt.figure(figsize=(20,20), dpi = 360)
sns.heatmap(corr, annot = True, fmt = '.2f')

##**Pipeline de Machine Learning**

In [ ]:
## Creamos un Pipeline para aplicar secuencialmente la lista de transformaciones definida previamente y un estimador final.
pipeline1 = Pipeline(
    steps=[("preprocesamiento",preprocesamiento),("estimador",SVC(probability=True))]
)

In [ ]:
## Definimos los hiperparámetros de los modelos a comparar.
parametros = [
    {    
        "estimador": (LogisticRegression(),),
          "estimador__C": (0.001, 1, 10, 100)
    },
     {
        "estimador": (SVC(probability=True),),
          "estimador__kernel":('rbf',), 
          "estimador__C":(0.1, 1, 10, 100, 1000), 
          "estimador__gamma":(0.001, 0.0001)      
    }
]

In [ ]:
## Creamos el Grid Search + Cross Validation.
grid_search1 = GridSearchCV(pipeline1, parametros,
                  refit = True,
                   cv = 5,
                   verbose=40)

In [ ]:
## Implementamos el GSCV con el set de train.
grid_search1.fit(xtrain_over,ytrain_over.ravel())

In [ ]:
## Guardamos en una variable toda la informacion del entrenamiento que quedó registrada en 'cv_results_'.
scores = grid_search1.cv_results_

## Imprimimos la informacion del entrenamiento.
scores_df = pd.DataFrame.from_dict(scores)
scores_df.head()

In [ ]:
## Imprimimos cual fue la mejor combinación de hiperparámetros.
print("Con un score de %0.4f, la mejor combinación de hiperparámtros fue: \n %s" % (grid_search1.best_score_, grid_search1.best_params_))

In [ ]:
## Obtenemos las predicciones a partir del modelo entrenado.
ypred = grid_search1.predict(xtest)

##**Mediciones de desempeño**##

In [ ]:
## Calculamos el accuracy.
test_acc = accuracy_score(ytest, ypred)
print("El accuracy es " + str(test_acc))

In [ ]:
## Computamos el área abajo de la curva ROC.
yproba = grid_search1.predict_proba(xtest)
fpr, tpr, thresholds = roc_curve(ytest.astype(int), yproba[:,1], drop_intermediate = False, pos_label=1)
auc_value = auc(fpr, tpr)
print("El AUC es = " + str(auc_value))

In [ ]:
## Imprimimos la curva ROC.
plt.figure(figsize=(7,7))
plt.plot(fpr, tpr, lw=2, alpha=0.7 , label = 'Curva ROC', color = 'b')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='r', alpha=.8)
plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=13)
plt.grid(False)
plt.legend(loc="lower right",fontsize=12)
plt.title('Curva ROC',fontsize=18)
plt.show()

In [ ]:
## Calculamos e imprimimos la Matriz de Confusión.
cm = confusion_matrix(ytest, ypred)
df_cm = pd.DataFrame(cm, index = ['No accede', 'Accede'], columns = ['No accede', "Accede"])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,fmt='g')
plt.title('Matriz de Confusión', fontsize=16)
plt.show()


##**Analisis de Componentes Pricipales**

In [ ]:
## Fijamos el porcentaje de varianza que deberán explicar los autovectores que se seleccionarán. 
n_comps = 0.8

## Definimos el PCA.
pca = PCA(n_components= n_comps)

In [ ]:
## Definimos las transformaciones a realizar para las variables numéricas. 
## Las estandarizaremos utilizando StandardScaler y luego aplicamos el método PCA.
numeric_transformer2 = Pipeline(
    steps=[("scaler", StandardScaler()), ("pca", pca)]
)

## Juntamos las transformaciones definidas previamente.
## Definimos el ColumnTransformer que será ejecutado al momento del fitting.
preprocesamiento2 = ColumnTransformer(
    transformers=[ 
        ("num", numeric_transformer2, num_features),
        ("cat", do_nothing, cat_features)
    ]
)

## Definimos un nuevo pipeline que incluye el método PCA en el preprosemiento.
pipeline2=Pipeline(
    steps=[('preprocesamiento',preprocesamiento2),('estimador',SVC())]
)

In [ ]:
## Observamos como queda el dataset que se obtine a partir de la reducción de la dimensionalidad.
pd.DataFrame(preprocesamiento2.fit_transform(xtrain_over))

In [ ]:
## Creamos el nuevo GSCV.
grid_search2 = GridSearchCV(pipeline2, parametros,
                  refit = True, 
                   cv = 5,
                   verbose=40)

In [ ]:
## Implementamos el GSCV.
grid_search2.fit(xtrain_over,ytrain_over.ravel())

In [ ]:
## Guardamos en una variable toda la informacion del entrenamiento que quedó registrada en 'cv_results_'.
scores2 = grid_search2.cv_results_

## Imprimimos la informacion del entrenamiento.
scores2_df = pd.DataFrame.from_dict(scores)
scores2_df.head()

In [ ]:
## Imprimimos cual fue el mejor modelo (estimator) y combinación de hiperparámetros.
print("Con un score de %0.4f, la mejor combinación de hiperparámtros fue: \n %s" % (grid_search2.best_score_, grid_search2.best_params_))

In [ ]:
## Obtenemos las predicciones a partir del modelo entrenado.
ypred2 = grid_search2.predict(xtest)

In [ ]:
## Calculamos el accuracy.
test_acc = accuracy_score(ytest, ypred2)
print("El accuracy es " + str(test_acc))

In [ ]:
## Computamos el área abajo de la curva ROC.
yproba2 = grid_search2.predict_proba(xtest)
fpr2, tpr2, thresholds = roc_curve(ytest.astype(int), yproba2[:,1], drop_intermediate = False, pos_label=1)
auc_value = auc(fpr2, tpr2)
print("El AUC es = " + str(auc_value))

In [ ]:
## Imprimimos la curva ROC.
plt.figure(figsize=(7,7))
plt.plot(fpr2, tpr2, lw=2, alpha=0.7 , label = 'Curva ROC', color = 'b')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='r', alpha=.8)
plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=13)
plt.grid(False)
plt.legend(loc="lower right", fontsize=12)
plt.title('Curva ROC', fontsize=16)
plt.show()

In [ ]:
## Calculamos e imprimimos la Matriz de Confusión.
cm = confusion_matrix(ytest, ypred2)
df_cm = pd.DataFrame(cm, index = ['No accede', 'Accede'], columns = ['No accede', "Accede"])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,fmt='g')
plt.title('Matriz de Confusión', fontsize=16)
plt.show()